<a href="https://colab.research.google.com/github/Kim-Yeon-Jun/Dacon/blob/main/Daegu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Dacon/Daegu_accident/df_train.csv")

In [ ]:
df2 = pd.read_csv("/content/drive/MyDrive/Dacon/Daegu_accident/df_test.csv")

In [ ]:
df.shape

(39601, 41)

In [ ]:
df2.shape

(10963, 37)

In [ ]:
df.drop(['구'], axis=1, inplace = True)

In [ ]:
df2.drop(['구'], axis = 1, inplace = True)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Unnamed: 0       10963 non-null  int64
 1   주말               10963 non-null  int64
 2   평일               10963 non-null  int64
 3   기타               10963 non-null  int64
 4   눈                10963 non-null  int64
 5   맑음               10963 non-null  int64
 6   비                10963 non-null  int64
 7   흐림               10963 non-null  int64
 8   남구               10963 non-null  int64
 9   달서구              10963 non-null  int64
 10  달성군              10963 non-null  int64
 11  동구               10963 non-null  int64
 12  북구               10963 non-null  int64
 13  서구               10963 non-null  int64
 14  수성구              10963 non-null  int64
 15  중구               10963 non-null  int64
 16  교차로 - 교차로부근      10963 non-null  int64
 17  교차로 - 교차로안       10963 non-null  int64
 18  교차로 - 

In [ ]:
df.drop(['Unnamed: 0'], inplace=True, axis = 1)
df2.drop(['Unnamed: 0'], inplace=True, axis = 1)

In [ ]:
# X_test = df2.copy()
X = df.drop(['사망자수','중상자수','경상자수','부상자수'], axis=1).copy()
y = df[['사망자수','중상자수','경상자수','부상자수']].copy()

In [ ]:
y.shape

(39601, 4)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =42)

In [ ]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size= 16, shuffle = True)

In [ ]:
class CustomModel(nn.Module):
  def __init__(self, input_size, output_size):
    super(CustomModel, self).__init__()
    self.fc1 = nn.Linear(input_size, 64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, output_size)

  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
input_size = 35
output_size = 4
model = CustomModel(input_size, output_size).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        outputs = model(x)

        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # 일정 간격마다 예측값 출력
    if (epoch + 1) % 10 == 0:
        model.eval()
        with torch.no_grad():
            inputs = torch.randn(5, input_size).to(device)  # 예측값을 확인할 임의의 입력 데이터
            predictions = torch.sigmoid(model(inputs))
            print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_dataloader):.4f}")
            print("Predictions:")
            print(predictions)



Epoch [10/100], Loss: 0.3302
Predictions:
tensor([[0.5042, 0.5750, 0.6668, 0.5564],
        [0.5149, 0.6461, 0.9223, 0.5063],
        [0.5110, 0.6332, 0.6618, 0.5097],
        [0.5272, 0.6137, 0.6588, 0.5244],
        [0.5422, 0.9053, 0.1533, 0.3407]], device='cuda:0')
Epoch [20/100], Loss: 0.3296
Predictions:
tensor([[0.4490, 0.5847, 0.9899, 0.4844],
        [0.5787, 0.4535, 0.3652, 0.9754],
        [0.5010, 0.5552, 0.7538, 0.5218],
        [0.6103, 0.3998, 0.1697, 0.9967],
        [0.5232, 0.5234, 0.6557, 0.7523]], device='cuda:0')
Epoch [30/100], Loss: 0.3290
Predictions:
tensor([[0.5050, 0.5522, 0.7966, 0.5458],
        [0.5291, 0.5384, 0.9558, 0.6587],
        [0.5194, 0.9048, 0.1373, 0.9258],
        [0.5473, 0.5279, 0.9874, 0.7341],
        [0.5012, 0.5543, 0.7504, 0.5276]], device='cuda:0')
Epoch [40/100], Loss: 0.3293
Predictions:
tensor([[0.4804, 0.4236, 0.6230, 0.9170],
        [0.4897, 0.3785, 0.5692, 0.8319],
        [0.5384, 0.7209, 0.0675, 0.4969],
        [0.4827, 0.430

In [ ]:
# 최종 학습 결과 확인
model.eval()
with torch.no_grad():
    inputs = torch.randn(5, input_size).to(device)  # 예측값을 확인할 임의의 입력 데이터
    predictions = torch.sigmoid(model(inputs))
    print("Final Predictions:")
    print(predictions)